# NASA EONET Data Visualization

This notebook creates visualizations using the Altair-Vega library for natural events data from NASA EONET.


In [ ]:
import pandas as pd
import altair as alt
from pathlib import Path
import os
from IPython.display import display, HTML

output_dir = "../graphs"

# Helper: print a clickable link to open an output file in the browser
def print_open_link(file_path, label=None):
    """Display a clickable link for `file_path` in Jupyter outputs."""
    try:
        abs_path = Path(file_path).resolve()
        display(HTML(f'<div>Saved: <code>{abs_path}</code> — <a href="file://{abs_path}" target="_blank">Open in browser</a></div>'))
    except Exception as e:
        # Fallback to simple print if display fails
        print(f"Saved: {file_path} (could not create clickable link: {e})")


## Load Data


In [14]:
# Load the processed events data
data_path = Path('../data/processed/events_2025.csv')

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data file not found: {data_path}. Please run data_pipeline.py first.")

df = pd.read_csv(data_path)

# Convert date column to datetime
df['occurrence_date'] = pd.to_datetime(df['occurrence_date'], errors='coerce')

# Filter to only keep records with year 2025
df = df[df['occurrence_date'].dt.year == 2025].copy()

# Filter out rows with missing dates for daily count chart
df_with_dates = df[df['occurrence_date'].notna()].copy()

print(f"Data loaded successfully!")
print(f"Total records (2025 only): {len(df)}")
print(f"Records with valid dates: {len(df_with_dates)}")
print(f"Date range: {df_with_dates['occurrence_date'].min()} to {df_with_dates['occurrence_date'].max()}")


Data loaded successfully!
Total records (2025 only): 3743
Records with valid dates: 3743
Date range: 2025-01-01 19:00:00+00:00 to 2025-11-12 12:00:00+00:00


## Data Preview


In [15]:
# Preview the first few rows of the dataframe
df.head()


,event_id,event_title,event_description,event_link,category_ids,category_titles,occurrence_date,occurrence_type,longitude,latitude,year,month,day
0,EONET_15859,Typhoon Fung-wong,NaN,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-06 00:00:00+00:00,Point,140.8,9.6,2025,11,6
1,EONET_15859,Typhoon Fung-wong,NaN,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-06 06:00:00+00:00,Point,140.6,9.7,2025,11,6
2,EONET_15859,Typhoon Fung-wong,NaN,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-06 12:00:00+00:00,Point,139.6,10.2,2025,11,6
3,EONET_15859,Typhoon Fung-wong,NaN,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-06 18:00:00+00:00,Point,138.6,10.7,2025,11,6
4,EONET_15859,Typhoon Fung-wong,NaN,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-07 00:00:00+00:00,Point,137.9,11.4,2025,11,7


## Daily Event Count Chart

Create a line/bar chart showing the number of events per day.


In [16]:
# Group by date and count events
# Normalize datetime to date (remove time component) but keep as datetime
df_with_dates['date'] = df_with_dates['occurrence_date'].dt.normalize()
daily_counts = df_with_dates.groupby('date').size().reset_index(name='event_count')

daily_counts




,date,event_count
0,2025-01-01 00:00:00+00:00,40
1,2025-01-02 00:00:00+00:00,30
2,2025-01-03 00:00:00+00:00,14
3,2025-01-04 00:00:00+00:00,20
4,2025-01-05 00:00:00+00:00,26
...,...,...
263,2025-11-08 00:00:00+00:00,4
264,2025-11-09 00:00:00+00:00,4
265,2025-11-10 00:00:00+00:00,4
266,2025-11-11 00:00:00+00:00,4


In [ ]:
# Create the chart
daily_chart = alt.Chart(daily_counts).mark_line(point=True).encode(
    x=alt.X('date:T', title='Date', axis=alt.Axis(format='%Y-%m-%d')),
    y=alt.Y('event_count:Q', title='Number of Events'),
    tooltip=['date:T', 'event_count:Q']
).properties(
    title='Daily Event Count',
    width=800,
    height=400
)

# Display the chart
daily_chart

# Save daily chart
daily_chart.save("../graphs/daily_event_count.html")
try:
    daily_chart.save(f"{output_dir}/daily_event_count.png", scale_factor=2)
    print(f"✓ Saved daily event count chart (HTML and PNG) to {output_dir}/")
except Exception as e:
    print(f"✓ Saved daily event count chart (HTML) to {output_dir}/")
    print(f"  Note: PNG export failed: {e}")

# Show clickable link to open the saved HTML
print_open_link(f"{output_dir}/daily_event_count.html")


✓ Saved daily event count chart (HTML) to ../graphs/
  Note: PNG export failed: Saving charts in 'png' format requires the vl-convert-python package: see https://altair-viz.github.io/user_guide/saving_charts.html#png-svg-and-pdf-format


## Geospatial Event Map

Create a world map with event locations plotted as points.


In [ ]:
from vega_datasets import data


# Filter out rows with missing coordinates
df_geo = df[(df['latitude'].notna()) & (df['longitude'].notna())].copy()

if df_geo.empty:
    print("Warning: No valid geographic data found for map visualization")
    map_chart = None
else:
    print(f"Creating map with {len(df_geo)} events with valid coordinates")
    
    # Create the geospatial map
    countries = alt.topo_feature(data.world_110m.url, 'countries')

    map_chart = alt.Chart(df_geo).mark_circle(
        size=50,
        opacity=0.6
    ).encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        color=alt.Color('category_titles:N', title='Event Category'),
        tooltip=[
            alt.Tooltip('event_title:N', title='Event'),
            alt.Tooltip('category_titles:N', title='Category'),
            alt.Tooltip('occurrence_date:T', title='Date'),
            alt.Tooltip('latitude:Q', title='Latitude', format='.2f'),
            alt.Tooltip('longitude:Q', title='Longitude', format='.2f')
        ]
    ).properties(
        title='Geospatial Event Map',
        width=1000,
        height=600
    ).project(
        type='naturalEarth1'
    )
    
   # Save map chart if it exists
if map_chart is not None:
    map_chart.save(f"{output_dir}/geospatial_event_map.html")
    try:
        map_chart.save(f"{output_dir}/geospatial_event_map.png", scale_factor=2)
        print(f"✓ Saved geospatial event map (HTML and PNG) to {output_dir}/")
    except Exception as e:
        print(f"✓ Saved geospatial event map (HTML) to {output_dir}/")
        print(f"  Note: PNG export failed: {e}")
    # Show clickable link to open the saved HTML
    print_open_link(f"{output_dir}/geospatial_event_map.html")
else:
    print("⚠ Skipping map chart (no valid geographic data)")

print("\nVisualization generation complete!")


Creating map with 3743 events with valid coordinates
✓ Saved geospatial event map (HTML) to ../graphs/
  Note: PNG export failed: Saving charts in 'png' format requires the vl-convert-python package: see https://altair-viz.github.io/user_guide/saving_charts.html#png-svg-and-pdf-format

Visualization generation complete!


# Radial Chart

In [19]:
df_categories = pd.read_csv("../data/processed/events_2025_category_counts.csv")

df_categories.head()

,category_title,event_count
0,Wildfires,4114
1,Sea and Lake Ice,192
2,Volcanoes,7


In [23]:
# ---- Load data ----
source = pd.DataFrame({
    "category_title": ["Wildfires", "Sea and Lake Ice", "Volcanoes"],
    "event_count": [4114, 192, 7]
})

# ---- Base chart ----
base = alt.Chart(source).encode(
    theta=alt.Theta("event_count:Q", stack=True),
    radius=alt.Radius(
        "event_count:Q",
        scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)
    ),
    color=alt.Color("category_title:N", title="Category")
)

# ---- Arc layer ----
c1 = base.mark_arc(innerRadius=20, stroke="#fff")

# ---- Label layer ----
c2 = base.mark_text(radiusOffset=10).encode(
    text="category_title:N"
)

# ---- Combine layers ----
radial_chart = (c1 + c2).properties(
    width=400,
    height=400,
    title="NASA Natural Events by Category)"
)

radial_chart.save(f"{output_dir}/nasa_events_radial_chart.html")